In [69]:
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
import re

driver_loc='/opt/homebrew/bin/chromedriver'

In [21]:
start_url = 'https://www.pgatour.com/schedule'
base_url = 'https://www.pgatour.com'
season = '2018-2019'



# Initiate webdriver
s = Service(driver_loc)
driver = webdriver.Chrome(service=s)
driver.get(start_url)

# Set view to full schedule
button_id = 'menu-button-:rr:'
try:
    WebDriverWait(driver, 10).until(expected_conditions.visibility_of_element_located((By.ID, button_id)))
except NoSuchElementException:
    raise NoSuchElementException(f'Dropdown for view is NOT FOUND')
view_button = driver.find_element('id', button_id)
view_button.click()

button_id = 'menu-list-:rr:-menuitem-:ru:'
try:
    WebDriverWait(driver, 10).until(expected_conditions.visibility_of_element_located((By.ID, button_id)))
except NoSuchElementException:
    raise NoSuchElementException(f'Dropdown for select view is NOT FOUND')
full_schedule_button = driver.find_element('id', button_id)
full_schedule_button.click()

## Set season to desired season
# Select dropdown
button_id = 'menu-button-:rd:'
try:
    WebDriverWait(driver, 10).until(expected_conditions.visibility_of_element_located((By.ID, button_id)))
except NoSuchElementException:
    raise NoSuchElementException(f'Dropdown for select season is NOT FOUND')

season_button = driver.find_element('id', button_id)
season_button.click()

# Read season options
soup = BeautifulSoup(driver.page_source, 'lxml')
season_buttons = soup.find(attrs={'id': 'menu-list-:rd:'}).find_all('button')
buttons = {button.text: button.get('id') for button in season_buttons}
"""buttons is of form {year: id}"""

# Confirm season is in buttons
if season not in buttons.keys():
    raise IndexError(f'Season "{season}" is not an option for this page.')

# Select the correct season
id_ = buttons[season]
select_season_button = driver.find_element('id', id_)
select_season_button.click()

#-------------------------------------
# Everything above is tested and works
#-------------------------------------

# Get tournament name and links from each tournament in view


# Final closeout
soup = BeautifulSoup(driver.page_source, 'lxml')
# driver.quit()

The above code works for: 
- setting the view to full season
- setting the season to desired season

In [58]:
class_1 = 'css-k8jakh'  # top level class
class_2 = 'css-yor8sx'  # 2nd level class
class_3 = 'css-0'       # 3rd level class
class_4 = 'chakra-text css-vgdvwe'  # 4th level class

tourneys = []
for whole_season in soup.find(attrs={'class': class_1}).find_all(attrs={'class': class_2}):
    for tourney in whole_season.find_all(attrs={'class': class_3}):
        tourney_dict = {}   # initialize new tourney_dict

        # Attempt to get link
        try:
            tourney_link = tourney.find_all('a', href=True)[0]['href']
            if not re.search(r'^/tournaments/', tourney_link):
                tourney_dict['tournament_link'] = None
            else:
                tourney_dict['tournament_link'] = base_url + tourney_link
        except (IndexError, AttributeError):
            tourney_dict['tournament_link'] = None

        # Attempt to get tournament name
        try:
            tourney_dict['tournament_name'] = tourney.find(attrs={'class': class_4}).text
        except (IndexError, AttributeError):
            tourney_dict['tournament_name'] = None

        # Pass if all values are none
        if not any(tourney_dict.values()):
            continue
        tourneys.append(tourney_dict)
    

In [33]:
from importlib import reload
import time
import json
import pga_data.mapping as mapping
reload(mapping)

mp = mapping.shot_make_meta()
seasons = [
           '2022-2023', 
           '2021-2022', 
           '2020-2021', 
           '2019-2020', 
           '2018-2019', 
           '2017-2018', 
           '2016-2017', 
           '2015-2016', 
           '2014-2015', 
           '2013-2014', 
           '2013', 
        #    '2012'   # 2012 doesnt seem to load
        ]

## Make the tourney_links files
tourney_links = mp.make_tourney_links(seasons, create_file=False)

In [35]:
from pathlib import Path
import os

base_path = Path('pga_data/data_files/')
file = base_path / 'tourney_links.json'
with open(file, 'w') as f:
    json.dump(tourney_links, f, indent=6)
    print(f'{os.path.split(file)[-1]} has been created!')


tourney_links.json has been created!
